# Практическое задание 1
**Определение любимого напитка человека методом _k_ ближайших соседей**

1. Конвертируем полученные с помощью опроса данные из XLSX в CSV при помощи https://cloudconvert.com/xlsx-to-csv

Выберем первые 5 элементов (голосов):

In [157]:
import pandas as pd
import math as m
try:
    data = pd.read_csv('./tea-vs-coffee.csv', engine='python')
except:
    data = pd.read_csv('https://github.com/dmittrj/AIST_Practice_1/blob/main/tea-vs-coffee.csv', engine='python')
data.head(10)

,Опрашиваемый,Укажите Ваш пол,Какой напиток Вы предпочитаете утром?,Укажите Ваш возраст,Насколько здоровый образ жизни Вы ведете? \nУкажите число по шкале от 0 до 100,Курите ли Вы?,Укажите цвет Вашего левого глаза,Много ли Вы испытываете стресса в жизни? Укажите число от 0 до 100,Вы высыпаетесь?,Укажите Ваш хронотип,"Во сколько Вы обычно просыпаетесь? Укажите время в формате ""Час"" и ""Минуты""",Сколько Вы в среднем спите? Укажите среди время вашего сна,Если поблизости с Вашим домом кофейня?,Вы являетесь гурманом?,Вы работаете из офиса?,Вы домосед?,У Вас есть хронические заболевания?,Какой рукой Вы пишите?,Какой у Вас знак зодиака?
0,1,Женщина,Кофе,22,65,Да,Серо-зеленый,55,Да,Жаворонок,8:00,8.0,Да,Да,Да,Нет,Нет,Правой,Козерог
1,2,Мужчина,Чай,21,85,Нет,Зеленый,70,Да,Жаворонок,6:00,8.0,Да,Нет,Нет,Нет,Нет,Левой,Стрелец
2,3,Мужчина,Кофе,22,50,Да,Голубой,70,Да,Сова,9:00,7.0,Да,Да,Нет,Да,Нет,Правой,Рак
3,4,Мужчина,Кофе,22,80,Да,Карий,60,Да,Сова,9:00,6.0,Да,Да,Нет,Нет,Да,Правой,Скорпион
4,5,Женщина,Кофе,23,50,Нет,Голубой,90,Нет,Сова,14:00,6.0,Нет,Нет,Нет,Да,Да,Левой,Скорпион
5,6,Мужчина,Чай,21,99,Нет,Зелёный,1,Да,Жаворонок,6:30,8.0,Нет,Да,Нет,Нет,Нет,Правой,Овен
6,7,Мужчина,Чай,22,50,Нет,Голубой,70,Да,Сова,10:00,6.0,Нет,Нет,Нет,Нет,Нет,Правой,Рыбы
7,8,Мужчина,Чай,22,80,Нет,Голубой,45,Да,Сова,7:00,7.0,Да,Нет,Да,Нет,Да,Правой,Водолей
8,9,Мужчина,Кофе,22,50,Нет,Коричневый,70,Нет,Сова,0:00,8.0,Да,Да,Нет,Да,Нет,Левой,Дева
9,10,Мужчина,Чай,21,70,Да,Зеленый,100,Нет,Сова,0:30,6.0,Нет,Нет,Нет,Да,Нет,Правой,Водолей


2. Для удобства работы переименуем столбцы датасета:

In [158]:
data.columns = ['reviewee', 'gender', 'drink', 'age', 'healthy_lifestyle', 'smoking', 'eye_color',
       'stressful', 'enough_sleep', 'lark_or_owl', 'waking_up', 'sleep_time',
       'cafe_nearby', 'gourmet', 'office_job', 'homester',
       'diseases', 'working_hand', 'zodiac_sign']
data.head(5)

,reviewee,gender,drink,age,healthy_lifestyle,smoking,eye_color,stressful,enough_sleep,lark_or_owl,waking_up,sleep_time,cafe_nearby,gourmet,office_job,homester,diseases,working_hand,zodiac_sign
0,1,Женщина,Кофе,22,65,Да,Серо-зеленый,55,Да,Жаворонок,8:00,8.0,Да,Да,Да,Нет,Нет,Правой,Козерог
1,2,Мужчина,Чай,21,85,Нет,Зеленый,70,Да,Жаворонок,6:00,8.0,Да,Нет,Нет,Нет,Нет,Левой,Стрелец
2,3,Мужчина,Кофе,22,50,Да,Голубой,70,Да,Сова,9:00,7.0,Да,Да,Нет,Да,Нет,Правой,Рак
3,4,Мужчина,Кофе,22,80,Да,Карий,60,Да,Сова,9:00,6.0,Да,Да,Нет,Нет,Да,Правой,Скорпион
4,5,Женщина,Кофе,23,50,Нет,Голубой,90,Нет,Сова,14:00,6.0,Нет,Нет,Нет,Да,Да,Левой,Скорпион


3. Приведем значения датасета в более удобные для дальнейшей обработки значения:

Бинарные значения:  
* gender: женщина = 0; мужчина = 1
* smoking: нет = 0; да = 1
* enough_sleep: нет = 0; да = 1
* lark_or_owl: жаворонок = 0; сова = 1
* cafe_nearby: нет = 0; да = 1
* gourmet: нет = 0; да = 1
* office_job: нет = 0; да = 1
* homester: нет = 0; да = 1
* diseases: нет = 0; да = 1
* working_hand: левой = 0; правой = 1

In [159]:
data["gender"] = data["gender"].replace("Женщина", 0)
data["gender"] = data["gender"].replace("Мужчина", 1)

data["smoking"] = data["smoking"].replace("Да", 1)
data["smoking"] = data["smoking"].replace("Нет", 0)

data["enough_sleep"] = data["enough_sleep"].replace("Да", 1)
data["enough_sleep"] = data["enough_sleep"].replace("Нет", 0)

data["lark_or_owl"] = data["lark_or_owl"].replace("Жаворонок", 0)
data["lark_or_owl"] = data["lark_or_owl"].replace("Сова", 1)

data["cafe_nearby"] = data["cafe_nearby"].replace("Да", 1)
data["cafe_nearby"] = data["cafe_nearby"].replace("Нет", 0)

data["gourmet"] = data["gourmet"].replace("Да", 1)
data["gourmet"] = data["gourmet"].replace("Нет", 0)

data["office_job"] = data["office_job"].replace("Да", 1)
data["office_job"] = data["office_job"].replace("Нет", 0)

data["homester"] = data["homester"].replace("Да", 1)
data["homester"] = data["homester"].replace("Нет", 0)

data["diseases"] = data["diseases"].replace("Да", 1)
data["diseases"] = data["diseases"].replace("Нет", 0)

data["working_hand"] = data["working_hand"].replace("Левой", 0)
data["working_hand"] = data["working_hand"].replace("Правой", 1)

data.head(5)

,reviewee,gender,drink,age,healthy_lifestyle,smoking,eye_color,stressful,enough_sleep,lark_or_owl,waking_up,sleep_time,cafe_nearby,gourmet,office_job,homester,diseases,working_hand,zodiac_sign
0,1,0,Кофе,22,65,1,Серо-зеленый,55,1,0,8:00,8.0,1,1,1,0,0,1,Козерог
1,2,1,Чай,21,85,0,Зеленый,70,1,0,6:00,8.0,1,0,0,0,0,0,Стрелец
2,3,1,Кофе,22,50,1,Голубой,70,1,1,9:00,7.0,1,1,0,1,0,1,Рак
3,4,1,Кофе,22,80,1,Карий,60,1,1,9:00,6.0,1,1,0,0,1,1,Скорпион
4,5,0,Кофе,23,50,0,Голубой,90,0,1,14:00,6.0,0,0,0,1,1,0,Скорпион


Рассмотрим небинарное значение цвета:

In [160]:
data["eye_color"]

0                Серо-зеленый
1                     Зеленый
2                     Голубой
3                       Карий
4                     Голубой
5                     Зелёный
6                     Голубой
7                     Голубой
8                 Коричневый 
9                     Зеленый
10                    Голубой
11                      Карий
12                   Болотный
13               Серо-зеленый
14                      Карий
15                    Голубой
16                      Серый
17               Серо-зеленый
18                      Серый
19                   Голубой 
20                      Синий
21                      Карий
22                      Карий
23                    Зелёный
24                      Карий
25                    Зелёный
26                      Синий
27                      карий
28                      Серый
29                    Зелёный
30                    Зелёный
31                      Серый
32               Серо-голубой
33        

Здесь можно отметить следующие проблемы:
* Некоторые семантически одинаковые значения синтаксически отличаются ("Зеленый" и "Зелёный", "Карий" и "Коричневый")
* Сложные и составные значения ("Серо-зеленый", "Болотный")
* Шуточные и не относящиеся к цвету значения ("Такой же, как и правого")

В моей работе цвета кодируются кортежем из двух значений:
1) Гамма (оттенок) цвета, определяется по спектру: красный (1), оранжевый (2), желтый (3), салатовый (4), зеленый (5), бирюзовый (6), голубой (7), синий (8), фиолетовый (9), розовый (10). Спектр замкнут по кругу (расстояние между 1 и 10 равно 1, а не 9)
2) Серость цвета. 0, если цвет чистый, 1, если цвет имеет серый оттенок.

Примечание. Карий (коричневый) цвет является комбинацией (2, 1)

In [161]:
data["eye_color"] = data["eye_color"].str.lower()
data["eye_color"] = data["eye_color"].str.strip()
data["eye_color"] = data["eye_color"].replace("зелёный", "зеленый")
data["eye_color"] = data["eye_color"].replace("коричневый", "карий")

def check_conditions(eye_color):
    spectrum = 3
    gray = 0
    if 'кари' in eye_color:
        spectrum = 2
        gray = 1
    elif 'зелен' in eye_color:
        spectrum = 5
    elif 'болот' in eye_color:
        spectrum = 5
        gray = 1
    elif 'голуб' in eye_color:
        spectrum = 7
    elif 'син' in eye_color:
        spectrum = 8
    if 'серо' in eye_color:
        gray = 1
    return f"{spectrum}, {gray}"

data['eye_color_2'] = data['eye_color'].apply(check_conditions)

data.head(38)

,reviewee,gender,drink,age,healthy_lifestyle,smoking,eye_color,stressful,enough_sleep,lark_or_owl,waking_up,sleep_time,cafe_nearby,gourmet,office_job,homester,diseases,working_hand,zodiac_sign,eye_color_2
0,1,0,Кофе,22,65,1,серо-зеленый,55,1,0,8:00,8.0,1,1,1,0,0,1,Козерог,"5, 1"
1,2,1,Чай,21,85,0,зеленый,70,1,0,6:00,8.0,1,0,0,0,0,0,Стрелец,"5, 0"
2,3,1,Кофе,22,50,1,голубой,70,1,1,9:00,7.0,1,1,0,1,0,1,Рак,"7, 0"
3,4,1,Кофе,22,80,1,карий,60,1,1,9:00,6.0,1,1,0,0,1,1,Скорпион,"2, 1"
4,5,0,Кофе,23,50,0,голубой,90,0,1,14:00,6.0,0,0,0,1,1,0,Скорпион,"7, 0"
5,6,1,Чай,21,99,0,зеленый,1,1,0,6:30,8.0,0,1,0,0,0,1,Овен,"5, 0"
6,7,1,Чай,22,50,0,голубой,70,1,1,10:00,6.0,0,0,0,0,0,1,Рыбы,"7, 0"
7,8,1,Чай,22,80,0,голубой,45,1,1,7:00,7.0,1,0,1,0,1,1,Водолей,"7, 0"
8,9,1,Кофе,22,50,0,карий,70,0,1,0:00,8.0,1,1,0,1,0,0,Дева,"2, 1"
9,10,1,Чай,21,70,1,зеленый,100,0,1,0:30,6.0,0,0,0,1,0,1,Водолей,"5, 0"


Особого внимания требуют опрашиваемые №№ 36 и 37. № 36 было принято выдать цвет (6, 1), а № 37 статистически самый частый

In [162]:
data.loc[data['reviewee'] == 37, 'eye_color_2'] = "6, 2"
data.loc[data['reviewee'] == 38, 'eye_color_2'] = data['eye_color_2'].value_counts().index[0]

data.tail(4)

,reviewee,gender,drink,age,healthy_lifestyle,smoking,eye_color,stressful,enough_sleep,lark_or_owl,waking_up,sleep_time,cafe_nearby,gourmet,office_job,homester,diseases,working_hand,zodiac_sign,eye_color_2
34,35,1,Чай,21,75,0,зеленый,50,1,0,8:00,8.0,1,0,1,1,0,1,Овен,"5, 0"
35,36,1,Кофе,27,60,1,зеленый,40,0,1,6:30,5.0,1,1,1,0,0,1,Близнецы,"5, 0"
36,37,1,Чай,21,65,1,зелёно-серо-голубой,75,0,1,10:15,5.5,0,1,0,1,0,1,Водолей,"6, 2"
37,38,1,Чай,21,80,0,"такой же, как и правого",70,1,1,10:00,10.0,0,1,0,0,1,1,Скорпион,"5, 0"


Столбец "zodiac_sign" следует привести к строчным буквам:

In [163]:
data["zodiac_sign"] = data["zodiac_sign"].str.lower()
data.head(7)

,reviewee,gender,drink,age,healthy_lifestyle,smoking,eye_color,stressful,enough_sleep,lark_or_owl,waking_up,sleep_time,cafe_nearby,gourmet,office_job,homester,diseases,working_hand,zodiac_sign,eye_color_2
0,1,0,Кофе,22,65,1,серо-зеленый,55,1,0,8:00,8.0,1,1,1,0,0,1,козерог,"5, 1"
1,2,1,Чай,21,85,0,зеленый,70,1,0,6:00,8.0,1,0,0,0,0,0,стрелец,"5, 0"
2,3,1,Кофе,22,50,1,голубой,70,1,1,9:00,7.0,1,1,0,1,0,1,рак,"7, 0"
3,4,1,Кофе,22,80,1,карий,60,1,1,9:00,6.0,1,1,0,0,1,1,скорпион,"2, 1"
4,5,0,Кофе,23,50,0,голубой,90,0,1,14:00,6.0,0,0,0,1,1,0,скорпион,"7, 0"
5,6,1,Чай,21,99,0,зеленый,1,1,0,6:30,8.0,0,1,0,0,0,1,овен,"5, 0"
6,7,1,Чай,22,50,0,голубой,70,1,1,10:00,6.0,0,0,0,0,0,1,рыбы,"7, 0"


Посмотрим, нет ли "шутников", голосовавших за несуществующие знаки зодиака

In [164]:
data.loc[(data['zodiac_sign'] != 'овен') & (data['zodiac_sign'] != 'телец') & (data['zodiac_sign'] != 'близнецы') & (data['zodiac_sign'] != 'рак') & (data['zodiac_sign'] != 'дева') & (data['zodiac_sign'] != 'лев') & (data['zodiac_sign'] != 'скорпион') & (data['zodiac_sign'] != 'весы') & (data['zodiac_sign'] != 'козерог') & (data['zodiac_sign'] != 'стрелец') & (data['zodiac_sign'] != 'водолей') & (data['zodiac_sign'] != 'рыбы'), 'zodiac_sign']

Series([], Name: zodiac_sign, dtype: object)

Таких нету.

4. Нормирование значений. Поле gender содержит значения от 0 до 1, в то время как поле healthy_lifestyle содержит значения от 0 до 100. Так, даже незначительные колебания здорового образа жизни оказывают большее влияние, чем пол  
Произведем нормирование полей age, helthy_lifestyle и stressful до значений от 0 до 1, sleeptime до значений от 0 до 2

In [165]:
max_val = data['age'].max()
normed_col1 = data['age'] / max_val
data['age'] = normed_col1

normed_col1 = data['healthy_lifestyle'] / 100
data['healthy_lifestyle'] = normed_col1

normed_col1 = data['stressful'] / 100
data['stressful'] = normed_col1

max_val = data['sleep_time'].max()
normed_col1 = data['sleep_time'] / max_val * 2
data['sleep_time'] = normed_col1

data.head(5)

,reviewee,gender,drink,age,healthy_lifestyle,smoking,eye_color,stressful,enough_sleep,lark_or_owl,waking_up,sleep_time,cafe_nearby,gourmet,office_job,homester,diseases,working_hand,zodiac_sign,eye_color_2
0,1,0,Кофе,0.814815,0.65,1,серо-зеленый,0.55,1,0,8:00,1.6,1,1,1,0,0,1,козерог,"5, 1"
1,2,1,Чай,0.777778,0.85,0,зеленый,0.70,1,0,6:00,1.6,1,0,0,0,0,0,стрелец,"5, 0"
2,3,1,Кофе,0.814815,0.50,1,голубой,0.70,1,1,9:00,1.4,1,1,0,1,0,1,рак,"7, 0"
3,4,1,Кофе,0.814815,0.80,1,карий,0.60,1,1,9:00,1.2,1,1,0,0,1,1,скорпион,"2, 1"
4,5,0,Кофе,0.851852,0.50,0,голубой,0.90,0,1,14:00,1.2,0,0,0,1,1,0,скорпион,"7, 0"


Рассмотрим значения столбца waking_up

In [166]:
data["waking_up"]

0      8:00
1      6:00
2      9:00
3      9:00
4     14:00
5      6:30
6     10:00
7      7:00
8      0:00
9      0:30
10     9:30
11     6:00
12    10:25
13    10:00
14     7:30
15    10:30
16     6:00
17     6:00
18    10:20
19     9:00
20     8:43
21     8:00
22     0:10
23     6:00
24     7:00
25     8:30
26     7:00
27     7:25
28     8:00
29     8:00
30     8:00
31     8:00
32     7:00
33     0:00
34     8:00
35     6:30
36    10:15
37    10:00
Name: waking_up, dtype: object

Удобно, что все указали время в формате ЧЧ:ММ. Самое позднее время - 14:00. Конвертируем значения времени в минуты с начала полуночи по формуле:  
$$
Время = часы * 60 + минуты
$$

In [167]:
def convert_time_to_minutes(time_str):
    hours, minutes = time_str.split(':')
    total_minutes = int(hours) * 60 + int(minutes)
    return total_minutes

data['waking_up'] = data['waking_up'].apply(convert_time_to_minutes)
data.head(5)

,reviewee,gender,drink,age,healthy_lifestyle,smoking,eye_color,stressful,enough_sleep,lark_or_owl,waking_up,sleep_time,cafe_nearby,gourmet,office_job,homester,diseases,working_hand,zodiac_sign,eye_color_2
0,1,0,Кофе,0.814815,0.65,1,серо-зеленый,0.55,1,0,480,1.6,1,1,1,0,0,1,козерог,"5, 1"
1,2,1,Чай,0.777778,0.85,0,зеленый,0.70,1,0,360,1.6,1,0,0,0,0,0,стрелец,"5, 0"
2,3,1,Кофе,0.814815,0.50,1,голубой,0.70,1,1,540,1.4,1,1,0,1,0,1,рак,"7, 0"
3,4,1,Кофе,0.814815,0.80,1,карий,0.60,1,1,540,1.2,1,1,0,0,1,1,скорпион,"2, 1"
4,5,0,Кофе,0.851852,0.50,0,голубой,0.90,0,1,840,1.2,0,0,0,1,1,0,скорпион,"7, 0"


Нормируем значения от 0 до 5:

In [171]:
max_val = data['waking_up'].max()
normed_col1 = data['waking_up'] / max_val * 5
data['waking_up'] = normed_col1

data.head(5)

,reviewee,gender,drink,age,healthy_lifestyle,smoking,eye_color,stressful,enough_sleep,lark_or_owl,waking_up,sleep_time,cafe_nearby,gourmet,office_job,homester,diseases,working_hand,zodiac_sign,eye_color_2
0,1,0,Кофе,0.814815,0.65,1,серо-зеленый,0.55,1,0,2.857143,1.6,1,1,1,0,0,1,козерог,"5, 1"
1,2,1,Чай,0.777778,0.85,0,зеленый,0.70,1,0,2.142857,1.6,1,0,0,0,0,0,стрелец,"5, 0"
2,3,1,Кофе,0.814815,0.50,1,голубой,0.70,1,1,3.214286,1.4,1,1,0,1,0,1,рак,"7, 0"
3,4,1,Кофе,0.814815,0.80,1,карий,0.60,1,1,3.214286,1.2,1,1,0,0,1,1,скорпион,"2, 1"
4,5,0,Кофе,0.851852,0.50,0,голубой,0.90,0,1,5.000000,1.2,0,0,0,1,1,0,скорпион,"7, 0"


Теперь данные в порядке.

5. Метод k ближайших соседей. Метод заключается в том, что объект относится к той группе, к которой он оказывается "ближе" в пространстве его свойств  
Ниже указаны значения k и размера обучающей группы train_amount.

In [206]:
k = 5
train_amount = 30

data_train = data.sample(n=train_amount)
data_train.columns = data.columns
data_test = data.drop(data_train.index)
data_test.columns = data.columns

Далее напишем функцию, которая будет определять степень похожести двух объектов - строк датфрейма

In [207]:
def compare_colors(color1, color2):
    c1_spectrum, c1_gray = color1.split(', ')
    c2_spectrum, c2_gray = color2.split(', ')

    spectrum_diff_1 = abs(int(c1_spectrum) - int(c2_spectrum))
    spectrum_diff_2 = abs(int(c1_spectrum) - 10 - int(c2_spectrum))
    spectrum_diff_3 = abs(int(c1_spectrum) - int(c2_spectrum) + 10)

    gray_diff = abs(int(c1_gray) - int(c2_gray))
    difference = min(spectrum_diff_1, spectrum_diff_2, spectrum_diff_3) + gray_diff
    return difference

def get_difference(row1, row2):
    age1 = int(row1['age'])
    age2 = int(row2['age'])

    gender1 = int(row1['gender'])
    gender2 = int(row2['gender'])

    hl1 = int(row1['healthy_lifestyle'])
    hl2 = int(row2['healthy_lifestyle'])

    smoking1 = int(row1['smoking'])
    smoking2 = int(row2['smoking'])

    stress1 = int(row1['stressful'])
    stress2 = int(row2['stressful'])

    sleep1 = int(row1['enough_sleep'])
    sleep2 = int(row2['enough_sleep'])

    lo1 = int(row1['lark_or_owl'])
    lo2 = int(row2['lark_or_owl'])

    wake_up1 = int(row1['waking_up'])
    wake_up2 = int(row2['waking_up'])

    sleep_time1 = int(row1['sleep_time'])
    sleep_time2 = int(row2['sleep_time'])

    cafe1 = int(row1['cafe_nearby'])
    cafe2 = int(row2['cafe_nearby'])

    gourmet1 = int(row1['gourmet'])
    gourmet2 = int(row2['gourmet'])

    office1 = int(row1['office_job'])
    office2 = int(row2['office_job'])

    homester1 = int(row1['homester'])
    homester2 = int(row2['homester'])

    diseases1 = int(row1['diseases'])
    diseases2 = int(row2['diseases'])

    wh1 = int(row1['working_hand'])
    wh2 = int(row2['working_hand'])

    colors = compare_colors(row1['eye_color_2'], row2['eye_color_2'])

    zodiac = 1 if row1['zodiac_sign'] == row2['zodiac_sign'] else 0
    
    difference = m.sqrt(pow(abs(age1 - age2), 2) + pow(abs(gender1 - gender2), 2) + pow(abs(hl1 - hl2), 2) + pow(abs(smoking1 - smoking2), 2) + pow(abs(stress1 - stress2), 2) + pow(abs(sleep1 - sleep2), 2) + pow(abs(lo1 - lo2), 2) + pow(abs(sleep_time1 - sleep_time2), 2) + pow(abs(cafe1 - cafe2), 2) + pow(abs(gourmet1 - gourmet2), 2) + pow(abs(office1 - office2), 2) + pow(abs(homester1 - homester2), 2) + pow(abs(diseases1 - diseases2), 2) + pow(abs(wh1 - wh2), 2) + pow(abs(wake_up1 - wake_up2), 2) + zodiac + pow(colors, 2))
    
    return difference

def compare_second(tup):
    return tup[1]

guessed = 0
for index_ts, row_ts in data_test.iterrows():
    neighbours = []
    for index_tr, row_tr in data_train.iterrows():
        diff = get_difference(row_ts, row_tr)
        if len(neighbours) < k:
            neighbours.append((row_tr["drink"], diff))
            neighbours = sorted(neighbours, key=compare_second)
        elif neighbours[-1][1] > diff:
            neighbours.remove(neighbours[-1])
            neighbours.append((row_tr["drink"], diff))
            neighbours = sorted(neighbours, key=compare_second)
    for_tea = 0
    for_coffee = 0
    for n in neighbours:
        if n[0] == "Чай":
            for_tea += 1
        else:
            for_coffee += 1
    decision = 'чай' if for_tea > for_coffee else 'кофе'
    guessed += (1 if decision == row_ts['drink'].lower() else 0)
    print(f"Метод k ({k}) ближайших соседей считает, что любимый напиток опрашиваемого №{row_ts['reviewee']} - {decision}. На самом деле: {row_ts['drink'].lower()}")
print(f"Всего отгадано: {guessed} из {38 - train_amount}. Точность: {guessed * 100 / (38 - train_amount)}%")


Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №2 - чай. На самом деле: чай
Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №4 - чай. На самом деле: кофе
Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №10 - чай. На самом деле: чай
Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №13 - чай. На самом деле: чай
Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №15 - чай. На самом деле: чай
Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №21 - чай. На самом деле: чай
Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №30 - чай. На самом деле: кофе
Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №31 - чай. На самом деле: чай
Всего отгадано: 6 из 8. Точность: 75.0%


Запустив несколько проверок, выясняется, что метод показывает точность от 50% до 75%. Попробум повысить его точность. Для этого воспользуемся следующими положениями:
* Сильнее всего на результат влияют значения smoking, enough_sleep, waking_up, sleep_time, cafe_nearby, diseases. Увеличим их вес вдвое
* Будем считать что параметры working_hand, zodiac_sign и eye_color не влияют никак. Не будем их учитывать

In [210]:
def get_difference_improved(row1, row2):
    age1 = int(row1['age'])
    age2 = int(row2['age'])

    gender1 = int(row1['gender'])
    gender2 = int(row2['gender'])

    hl1 = int(row1['healthy_lifestyle'])
    hl2 = int(row2['healthy_lifestyle'])

    smoking1 = int(row1['smoking'])
    smoking2 = int(row2['smoking'])

    stress1 = int(row1['stressful'])
    stress2 = int(row2['stressful'])

    sleep1 = int(row1['enough_sleep'])
    sleep2 = int(row2['enough_sleep'])

    lo1 = int(row1['lark_or_owl'])
    lo2 = int(row2['lark_or_owl'])

    wake_up1 = int(row1['waking_up'])
    wake_up2 = int(row2['waking_up'])

    sleep_time1 = int(row1['sleep_time'])
    sleep_time2 = int(row2['sleep_time'])

    cafe1 = int(row1['cafe_nearby'])
    cafe2 = int(row2['cafe_nearby'])

    gourmet1 = int(row1['gourmet'])
    gourmet2 = int(row2['gourmet'])

    office1 = int(row1['office_job'])
    office2 = int(row2['office_job'])

    homester1 = int(row1['homester'])
    homester2 = int(row2['homester'])

    diseases1 = int(row1['diseases'])
    diseases2 = int(row2['diseases'])
    
    difference = m.sqrt(pow(abs(age1 - age2), 2) + pow(abs(gender1 - gender2), 2) + pow(abs(hl1 - hl2), 2) + pow(abs(smoking1 - smoking2) * 2, 2) + pow(abs(stress1 - stress2), 2) + pow(abs(sleep1 - sleep2) * 2, 2) + pow(abs(lo1 - lo2), 2) + pow(abs(sleep_time1 - sleep_time2) * 2, 2) + pow(abs(cafe1 - cafe2) * 2, 2) + pow(abs(gourmet1 - gourmet2), 2) + pow(abs(office1 - office2), 2) + pow(abs(homester1 - homester2), 2) + pow(abs(diseases1 - diseases2) * 2, 2) + pow(abs(wake_up1 - wake_up2), 2))
    
    return difference

def compare_second(tup):
    return tup[1]

guessed = 0
for index_ts, row_ts in data_test.iterrows():
    neighbours = []
    for index_tr, row_tr in data_train.iterrows():
        diff = get_difference_improved(row_ts, row_tr)
        if len(neighbours) < k:
            neighbours.append((row_tr["drink"], diff))
            neighbours = sorted(neighbours, key=compare_second)
        elif neighbours[-1][1] > diff:
            neighbours.remove(neighbours[-1])
            neighbours.append((row_tr["drink"], diff))
            neighbours = sorted(neighbours, key=compare_second)
    for_tea = 0
    for_coffee = 0
    for n in neighbours:
        if n[0] == "Чай":
            for_tea += 1
        else:
            for_coffee += 1
    decision = 'чай' if for_tea > for_coffee else 'кофе'
    guessed += (1 if decision == row_ts['drink'].lower() else 0)
    print(f"Метод k ({k}) ближайших соседей считает, что любимый напиток опрашиваемого №{row_ts['reviewee']} - {decision}. На самом деле: {row_ts['drink'].lower()}")
print(f"Всего отгадано: {guessed} из {38 - train_amount}. Точность: {guessed * 100 / (38 - train_amount)}%")


Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №2 - чай. На самом деле: чай
Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №4 - кофе. На самом деле: кофе
Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №10 - чай. На самом деле: чай
Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №13 - чай. На самом деле: чай
Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №15 - чай. На самом деле: чай
Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №21 - чай. На самом деле: чай
Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №30 - кофе. На самом деле: кофе
Метод k (5) ближайших соседей считает, что любимый напиток опрашиваемого №31 - чай. На самом деле: чай
Всего отгадано: 8 из 8. Точность: 100.0%


Точность приблизилась к 100%.